In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Korak 1: Definirajte Pydantic modele za strukturirane izlaze

Ovi modeli definiraju **shemu** koju će agenti vratiti. Korištenje `response_format` s Pydanticom osigurava:
- ✅ Sigurno izvlačenje podataka prema tipu
- ✅ Automatsku validaciju
- ✅ Nema pogrešaka u parsiranju iz slobodnog teksta
- ✅ Jednostavno uvjetno usmjeravanje na temelju polja


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Korak 2: Izrada alata za rezervaciju hotela

Ovaj alat će **availability_agent** pozivati kako bi provjerio dostupnost soba. Koristimo dekorator `@ai_function` za:
- Pretvaranje Python funkcije u alat koji AI može pozvati
- Automatsko generiranje JSON sheme za LLM
- Obradu provjere parametara
- Omogućavanje automatskog poziva od strane agenata

Za ovu demonstraciju:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → Imaju sobe ✅
- **Svi ostali gradovi** → Nema soba ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Korak 3: Definirajte funkcije uvjeta za usmjeravanje

Ove funkcije pregledavaju odgovor agenta i određuju koji put slijediti u tijeku rada.

**Ključni uzorak:**
1. Provjerite je li poruka `AgentExecutorResponse`
2. Parsirajte strukturirani izlaz (Pydantic model)
3. Vratite `True` ili `False` za upravljanje usmjeravanjem

Tijek rada će procijeniti ove uvjete na **rubovima** kako bi odlučio kojeg izvršitelja pozvati sljedećim.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Korak 4: Kreirajte prilagođeni izvršitelj prikaza

Izvršitelji su komponenta tijeka rada koja obavlja transformacije ili sporedne efekte. Koristimo dekorator `@executor` za kreiranje prilagođenog izvršitelja koji prikazuje konačni rezultat.

**Ključni pojmovi:**
- `@executor(id="...")` - Registrira funkciju kao izvršitelja tijeka rada
- `WorkflowContext[Never, str]` - Tipizirani pokazatelji za ulaz/izlaz
- `ctx.yield_output(...)` - Iterira konačni rezultat tijeka rada


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Korak 5: Učitajte varijable okoline

Konfigurirajte LLM klijent. Ovaj primjer radi s:
- **GitHub modelima** (besplatna razina s GitHub tokenom)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Korak 6: Izrada AI agenata sa strukturiranim izlazima

Izrađujemo **tri specijalizirana agenta**, svaki umotan u `AgentExecutor`:

1. **availability_agent** - Provjerava dostupnost hotela koristeći alat
2. **alternative_agent** - Predlaže alternativne gradove (kad nema soba)
3. **booking_agent** - Potakne na rezervaciju (kad su sobe dostupne)

**Ključne značajke:**
- `tools=[hotel_booking]` - Dostavlja agentu alat
- `response_format=PydanticModel` - Nametne strukturirani JSON izlaz
- `AgentExecutor(..., id="...")` - Omotava agenta za upotrebu u tijeku rada


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Korak 7: Izgradnja toka rada s uvjetnim granama

Sada koristimo `WorkflowBuilder` za izgradnju grafa s uvjetnim usmjeravanjem:

**Struktura toka rada:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Ključne metode:**
- `.set_start_executor(...)` - Postavlja početnu točku
- `.add_edge(from, to, condition=...)` - Dodaje uvjetnu granu
- `.build()` - Završava tok rada


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Korak 8: Pokreni test slučaj 1 - Grad BEZ dostupnosti (Pariz)

Testirajmo put **bez dostupnosti** tražeći hotele u Parizu (koji nema soba u našoj simulaciji).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Korak 9: Pokreni Test Slučaj 2 - Grad S Dostatnošću (Stockholm)

Sada testirajmo putanju **dostupnosti** tražeći hotele u Stockholmu (koji ima sobe u našoj simulaciji).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Ključni zaključci i sljedeći koraci

### ✅ Što ste naučili:

1. **WorkflowBuilder uzorak**
   - Koristite `.set_start_executor()` za definiranje ulazne točke
   - Koristite `.add_edge(from, to, condition=...)` za uvjetno usmjeravanje
   - Pozovite `.build()` za završetak workflow-a

2. **Uvjetno usmjeravanje**
   - Funkcije za uvjete pregledavaju `AgentExecutorResponse`
   - Parsiraju strukturirane izlaze za donošenje odluka o usmjeravanju
   - Vraćaju `True` za aktiviranje veze, `False` za preskakanje

3. **Integracija alata**
   - Koristite `@ai_function` za pretvaranje Python funkcija u AI alate
   - Agent automatski poziva alate kad je potrebno
   - Alati vraćaju JSON koji agenti mogu parsirati

4. **Strukturirani izlazi**
   - Koristite Pydantic modele za tip-no sigurno izdvajanje podataka
   - Postavite `response_format=MyModel` pri kreiranju agenata
   - Parsirajte odgovore s `Model.model_validate_json()`

5. **Prilagođeni izvršitelji**
   - Koristite `@executor(id="...")` za kreiranje komponenti workflow-a
   - Izvršitelji mogu transformirati podatke ili obavljati sporedne radnje
   - Koristite `ctx.yield_output()` za proizvodnju rezultata workflow-a

### 🚀 Primjena u stvarnom svijetu:

- **Rezervacija putovanja**: Provjera dostupnosti, predlaganje alternativa, usporedba opcija
- **Korisnička podrška**: Usmjeravanje na temelju vrste problema, sentimenta, prioriteta
- **E-trgovina**: Provjera zaliha, predlaganje alternativa, obrada narudžbi
- **Moderacija sadržaja**: Usmjeravanje na temelju razine toksičnosti, oznaka korisnika
- **Workflow odobrenja**: Usmjeravanje prema iznosu, korisničkoj ulozi, razini rizika
- **Višestupanjska obrada**: Usmjeravanje na temelju kvalitete i potpunosti podataka

### 📚 Sljedeći koraci:

- Dodati složenije uvjete (više kriterija)
- Implementirati petlje s upravljanjem stanja workflow-a
- Dodati pod-workflow-e za ponovnu upotrebu komponenti
- Integrirati s pravim API-jima (rezervacije hotela, sustavi zaliha)
- Dodati rukovanje pogreškama i rezervne putove
- Vizualizirati workflow-e s ugrađenim alatima za vizualizaciju


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Izjava o odricanju od odgovornosti**:
Ovaj dokument je preveden korištenjem AI prevoditeljskog servisa [Co-op Translator](https://github.com/Azure/co-op-translator). Iako težimo točnosti, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za važne informacije preporučuje se profesionalni ljudski prijevod. Ne odgovaramo za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
